In [92]:
import requests
import pandas as pd

In [2]:
# Esta funcion imprime las 10 monedas volatiles con mas capitalizacion y luego las 10 monedas estables.
def get_top_coins():
    stable = []
    volatil = []

    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()

    for coin in data:
        if coin['current_price'] > 1 or 0.99 > coin['current_price'] and len(volatil) < 10:
            volatil.append(coin['id'])
        elif 1.1 > coin['current_price'] > 0.97 and len(stable) < 10:
            stable.append(coin['id'])
        else:
            break
        
    return stable,volatil

stable, volatil = get_top_coins()
print(stable)
print(volatil)

['tether', 'usd-coin']
['bitcoin', 'ethereum', 'binancecoin', 'ripple', 'staked-ether', 'cardano', 'dogecoin', 'solana', 'tron', 'polkadot']


In [185]:
# Utiliza una lista de nombres de criptomonedas y a traves de la api de coingecko extrae la informacion de cada una de ellas para luego de un ETL guardar 
# la informacion en un archivo csv.
def get_historical_data(coinnamelist,days):
    for coin in coinnamelist:
        url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
        params = {
            'vs_currency': 'usd',
            'days': days,
            'interval': 'daily'
        }
    
        response = requests.get(url, params=params)
    
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)
            df['market_caps'] = df['market_caps'].apply(lambda x: int(x[1]))
            df['total_volumes'] = df['total_volumes'].apply(lambda x: int(x[1]))
            df['Date'] = df['prices'].apply(lambda x: pd.to_datetime(x[0], unit='ms'))
            df['prices'] = df['prices'].apply(lambda x: int(x[1]))
            df.rename(columns={'prices': 'Price_usd','total_volumes':'Volume','market_caps':'Market_cap'}, inplace=True)
            df = df.groupby(df["Date"].dt.to_period("M")).tail(1)
            df.to_csv(f'./Data/{coin}.csv',index=False)
        else:
            print(f"Error al obtener los datos de {coin}:", response.status_code)


In [28]:
get_historical_data(volatil)

***

Obtencion de datos globales

In [3]:
url = 'https://api.coingecko.com/api/v3/global'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

In [4]:
data

{'data': {'active_cryptocurrencies': 10150,
  'upcoming_icos': 0,
  'ongoing_icos': 49,
  'ended_icos': 3376,
  'markets': 836,
  'total_market_cap': {'btc': 41909168.0563866,
   'eth': 659824072.8019023,
   'ltc': 16787531520.292736,
   'bch': 5725551987.714669,
   'bnb': 5107265646.735155,
   'eos': 1869087785530.507,
   'xrp': 2084736384445.723,
   'xlm': 8719990169319.429,
   'link': 174022950370.4369,
   'dot': 247451527525.28854,
   'yfi': 196650428.46162176,
   'usd': 1111275380341.9402,
   'aed': 4081753366634.2646,
   'ars': 388900709701547.44,
   'aud': 1715246881905.506,
   'bdt': 121618100976189.36,
   'bhd': 418990824302.6044,
   'bmd': 1111275380341.9402,
   'brl': 5400909475999.873,
   'cad': 1502434312743.8823,
   'chf': 975814245304.3989,
   'clp': 958819510912830.6,
   'cny': 7986736158517.532,
   'czk': 24635820567060.652,
   'dkk': 7622962385313.375,
   'eur': 1022662281513.4741,
   'gbp': 872999047115.1628,
   'hkd': 8713454693492.135,
   'huf': 391243091509044.3,


In [5]:
data = data['data']
df_global = pd.DataFrame(data)
df_global.head()

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10150,0,49,3376,836,4.190917e+07,1.946119e+06,46.447861,2.646764,1692812860
eth,10150,0,49,3376,836,6.598241e+08,3.063998e+07,18.243934,2.646764,1692812860
ltc,10150,0,49,3376,836,1.678753e+10,7.795558e+08,NaN,2.646764,1692812860
bch,10150,0,49,3376,836,5.725552e+09,2.658751e+08,NaN,2.646764,1692812860
bnb,10150,0,49,3376,836,5.107266e+09,2.371640e+08,3.017118,2.646764,1692812860


In [150]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, btc to sol
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   active_cryptocurrencies               67 non-null     int64  
 1   upcoming_icos                         67 non-null     int64  
 2   ongoing_icos                          67 non-null     int64  
 3   ended_icos                            67 non-null     int64  
 4   markets                               67 non-null     int64  
 5   total_market_cap                      61 non-null     float64
 6   total_volume                          61 non-null     float64
 7   market_cap_percentage                 10 non-null     float64
 8   market_cap_change_percentage_24h_usd  67 non-null     float64
 9   updated_at                            67 non-null     int64  
dtypes: float64(4), int64(6)
memory usage: 5.8+ KB


Faltan datos importante en las columnas total_market_cap y market_cap_percentage

In [6]:
df_global['market_cap_percentage'].sum()

83.74989519178826

La sumatoria de la columna market_cap_percentage (porcentaje de capitalizacion) no llega al 100 % o cercano.

In [12]:
df_global = df_global.sort_values(by='market_cap_percentage', ascending=False)
df_global.head()

AttributeError: 'list' object has no attribute 'sort_values'

***

Utilizando la opcion /coins/market de la api de coingecko para obtener datos mas completos.

In [148]:
# Esta funcion retorna los datos de todas las criptomonedas en coingecko
def all_coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 250,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()
        
    return data

In [149]:
df_global = all_coins()

In [150]:
df_global = pd.DataFrame(df_global)
df_global.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,25965.000000,505334532485,1,5.451093e+11,1.192175e+10,26200.000000,...,2.100000e+07,2.100000e+07,69045.00,-62.41928,2021-11-10T14:24:11.849Z,67.810000,38165.60610,2013-07-06T00:00:00.000Z,None,2023-08-25T20:08:16.581Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1645.390000,197834905602,2,1.978349e+11,7.262542e+09,1664.660000,...,1.202164e+08,NaN,4878.26,-66.26873,2021-11-10T14:24:19.604Z,0.432979,379941.41386,2015-10-20T00:00:00.000Z,"{'times': 83.7321993318686, 'currency': 'btc',...",2023-08-25T20:08:18.543Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999503,82779760569,3,8.277976e+10,1.122434e+10,1.002000,...,8.284913e+10,NaN,1.32,-24.48294,2018-07-24T00:00:00.000Z,0.572521,74.51983,2015-03-02T00:00:00.000Z,None,2023-08-25T20:05:00.524Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,217.310000,33421449941,4,4.344506e+10,3.896782e+08,219.050000,...,1.538562e+08,2.000000e+08,686.31,-68.35728,2021-05-10T07:24:17.097Z,0.039818,545300.82367,2017-10-19T00:00:00.000Z,None,2023-08-25T20:08:12.635Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.524097,27780175668,5,5.250042e+10,9.233069e+08,0.529816,...,9.998849e+10,1.000000e+11,3.40,-84.54826,2018-01-07T00:00:00.000Z,0.002686,19448.74830,2014-05-22T00:00:00.000Z,None,2023-08-25T20:08:18.814Z


In [151]:
df_global

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,25965.000000,505334532485,1,5.451093e+11,1.192175e+10,26200.000000,...,2.100000e+07,2.100000e+07,69045.000000,-62.41928,2021-11-10T14:24:11.849Z,67.810000,38165.60610,2013-07-06T00:00:00.000Z,None,2023-08-25T20:08:16.581Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1645.390000,197834905602,2,1.978349e+11,7.262542e+09,1664.660000,...,1.202164e+08,NaN,4878.260000,-66.26873,2021-11-10T14:24:19.604Z,0.432979,379941.41386,2015-10-20T00:00:00.000Z,"{'times': 83.7321993318686, 'currency': 'btc',...",2023-08-25T20:08:18.543Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999503,82779760569,3,8.277976e+10,1.122434e+10,1.002000,...,8.284913e+10,NaN,1.320000,-24.48294,2018-07-24T00:00:00.000Z,0.572521,74.51983,2015-03-02T00:00:00.000Z,None,2023-08-25T20:05:00.524Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,217.310000,33421449941,4,4.344506e+10,3.896782e+08,219.050000,...,1.538562e+08,2.000000e+08,686.310000,-68.35728,2021-05-10T07:24:17.097Z,0.039818,545300.82367,2017-10-19T00:00:00.000Z,None,2023-08-25T20:08:12.635Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.524097,27780175668,5,5.250042e+10,9.233069e+08,0.529816,...,9.998849e+10,1.000000e+11,3.400000,-84.54826,2018-01-07T00:00:00.000Z,0.002686,19448.74830,2014-05-22T00:00:00.000Z,None,2023-08-25T20:08:18.814Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,constellation-labs,dag,Constellation,https://assets.coingecko.com/coins/images/4645...,0.031662,91380897,246,1.122320e+08,2.902380e+05,0.033366,...,3.550000e+09,NaN,0.451761,-93.00191,2021-08-25T23:30:00.311Z,0.001102,2769.13057,2019-03-09T00:00:00.000Z,"{'times': -0.7113880320957678, 'currency': 'et...",2023-08-25T20:08:12.309Z
246,hxro,hxro,HXRO,https://assets.coingecko.com/coins/images/7805...,0.149332,91150254,247,1.493322e+08,1.533200e+03,0.149391,...,1.000000e+09,NaN,0.800981,-81.35633,2021-06-05T00:59:41.210Z,0.009147,1532.62149,2019-11-10T15:12:15.992Z,None,2023-08-24T23:11:11.075Z
247,nervos-network,ckb,Nervos Network,https://assets.coingecko.com/coins/images/9566...,0.002721,90786273,248,NaN,1.265748e+06,0.002753,...,NaN,NaN,0.043706,-93.76801,2021-03-31T17:08:59.155Z,0.001979,37.63930,2022-12-17T14:25:57.243Z,"{'times': -0.7278888111934925, 'currency': 'us...",2023-08-25T20:08:13.100Z
248,cartesi,ctsi,Cartesi,https://assets.coingecko.com/coins/images/1103...,0.121274,90315098,249,1.214656e+08,3.464750e+06,0.122575,...,1.000000e+09,1.000000e+09,1.740000,-93.03892,2021-05-09T00:43:41.306Z,0.026706,354.62782,2020-05-10T04:49:27.744Z,None,2023-08-25T20:08:17.874Z


In [152]:
df_global.describe()

,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,atl,atl_change_percentage
count,2.500000e+02,2.500000e+02,250.000000,2.390000e+02,2.500000e+02,2.470000e+02,2.470000e+02,247.000000,247.000000,2.480000e+02,248.000000,2.500000e+02,2.400000e+02,1.470000e+02,2.500000e+02,250.000000,250.000000,2.500000e+02
mean,5.386932e+02,4.258463e+09,125.500000,5.356790e+09,1.839595e+08,5.306559e+02,5.213720e+02,-1.713002,-0.557659,-7.971503e+06,-0.567340,6.364534e+14,1.764377e+15,2.873697e+15,5.510643e+05,-79.618075,129.387538,6.292360e+34
std,3.300174e+03,3.472000e+10,72.312977,3.801337e+10,1.159188e+09,3.343609e+03,3.288807e+03,18.946210,3.670578,7.730704e+07,3.715579,9.873927e+15,2.711019e+16,3.463985e+16,8.679827e+06,24.626585,881.144829,9.949094e+35
min,1.085000e-09,8.903418e+07,1.000000,8.903418e+07,0.000000e+00,1.115000e-09,1.049000e-09,-233.660875,-14.807790,-9.215733e+08,-15.434060,0.000000e+00,4.589498e+03,8.969497e+03,6.345000e-09,-100.000000,0.000000,-8.201900e-01
25%,1.385230e-01,1.495771e+08,63.250000,2.034593e+08,2.040430e+06,1.478920e-01,1.412990e-01,-0.027976,-1.557485,-5.053144e+06,-1.656378,8.252326e+07,1.058640e+08,1.000000e+08,1.220000e+00,-95.700598,0.012428,4.454140e+01
50%,7.053120e-01,2.532636e+08,125.500000,4.439432e+08,9.057332e+06,7.728370e-01,7.086510e-01,-0.000814,-0.305590,-1.335690e+06,-0.467725,4.490943e+08,1.000000e+09,1.000000e+09,6.230000e+00,-91.010215,0.193030,2.821820e+02
75%,4.757500e+00,6.029965e+08,187.750000,1.042977e+09,2.708986e+07,4.825000e+00,4.640000e+00,0.000500,0.372420,9.462818e+05,0.295488,2.960391e+09,4.494374e+09,4.647484e+09,5.765000e+01,-71.984945,0.893429,1.754448e+03
max,2.598600e+04,5.053345e+11,250.000000,5.451093e+11,1.192175e+10,2.631700e+04,2.583100e+04,121.790000,26.784300,4.708793e+08,26.520940,1.561249e+17,4.200000e+17,4.200000e+17,1.372421e+08,-2.534340,10704.550000,1.573090e+37


In [153]:
df_global.columns

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')

Comprobando la esactitud calculando el porcentaje de capitalizacion o dominancia de bitcoin con los datos provistos.

In [154]:
df_global[df_global['id'] == 'bitcoin']['market_cap'][0]/df_global['market_cap'].sum()

0.4746638207758425

Segun https://coin360.com/ los datos son correctos.

Eliminando columnas innecesarias

In [155]:
df_global.drop(['image','high_24h','ath_date','atl_date','low_24h','price_change_24h','price_change_percentage_24h','market_cap_change_24h','market_cap_change_percentage_24h','roi','ath_change_percentage','atl_change_percentage','total_supply','max_supply'],axis=1,inplace=True)

In [156]:
df_global.head()

,id,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,circulating_supply,ath,atl,last_updated
0,bitcoin,btc,Bitcoin,25965.000000,505334532485,1,5.451093e+11,1.192175e+10,1.946770e+07,69045.00,67.810000,2023-08-25T20:08:16.581Z
1,ethereum,eth,Ethereum,1645.390000,197834905602,2,1.978349e+11,7.262542e+09,1.202164e+08,4878.26,0.432979,2023-08-25T20:08:18.543Z
2,tether,usdt,Tether,0.999503,82779760569,3,8.277976e+10,1.122434e+10,8.284913e+10,1.32,0.572521,2023-08-25T20:05:00.524Z
3,binancecoin,bnb,BNB,217.310000,33421449941,4,4.344506e+10,3.896782e+08,1.538562e+08,686.31,0.039818,2023-08-25T20:08:12.635Z
4,ripple,xrp,XRP,0.524097,27780175668,5,5.250042e+10,9.233069e+08,5.291419e+10,3.40,0.002686,2023-08-25T20:08:18.814Z


In [157]:
df_global.rename(columns={'ath': 'max_price',
                   'atl': 'min_price',
                   }, inplace=True)

In [158]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       250 non-null    object 
 1   symbol                   250 non-null    object 
 2   name                     250 non-null    object 
 3   current_price            250 non-null    float64
 4   market_cap               250 non-null    int64  
 5   market_cap_rank          250 non-null    int64  
 6   fully_diluted_valuation  239 non-null    float64
 7   total_volume             250 non-null    float64
 8   circulating_supply       250 non-null    float64
 9   max_price                250 non-null    float64
 10  min_price                250 non-null    float64
 11  last_updated             250 non-null    object 
dtypes: float64(6), int64(2), object(4)
memory usage: 23.6+ KB


El dataframe no contiene datos nulos o vacios.

In [134]:
df_global.to_csv('./Data/all_coins.csv',index=False)

***

Extrayendo informacion de exchanges

In [103]:
def get_exchange_info(limit=None, base_currency=None):
    base_url = "https://api.coingecko.com/api/v3/exchanges"
    params = {'per_page': 250}
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        exchanges_data = response.json()
        return exchanges_data
    else:
        print("Error:", response.status_code)
        return None

In [104]:
df_exchanges = get_exchange_info()

In [105]:
df_exchanges = pd.DataFrame(df_exchanges)
df_exchanges

,id,name,year_established,country,description,url,image,has_trading_incentive,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,binance,Binance,2017.0,Cayman Islands,,https://www.binance.com/,https://assets.coingecko.com/markets/images/52...,False,10.0,1.0,200455.723795,131112.818914
1,gdax,Coinbase Exchange,2012.0,United States,,https://coinbase-consumer.sjv.io/coingecko,https://assets.coingecko.com/markets/images/23...,False,10.0,2.0,30308.933223,30308.933223
2,bybit_spot,Bybit,2018.0,British Virgin Islands,Bybit is a cryptocurrency exchange that offers...,https://www.bybit.com,https://assets.coingecko.com/markets/images/69...,False,10.0,3.0,28728.843946,28728.843946
3,huobi,Huobi,2013.0,Seychelles,,https://www.huobi.com,https://assets.coingecko.com/markets/images/25...,False,10.0,4.0,38706.507138,19743.017826
4,kraken,Kraken,2011.0,United States,,https://r.kraken.com/c/2223866/687155/10583,https://assets.coingecko.com/markets/images/29...,False,10.0,5.0,18114.272692,13082.623548
...,...,...,...,...,...,...,...,...,...,...,...,...
245,verse,Verse,2022.0,None,"The Verse DEX, available at <a href=""http://ve...",https://verse.bitcoin.com/,https://assets.coingecko.com/markets/images/98...,False,5.0,245.0,0.342037,0.342037
246,polycat_finance,Polycat Finance,NaN,None,Polycat launched May 2021 as a yield farm and ...,https://polycat.finance/swap,https://assets.coingecko.com/markets/images/77...,False,5.0,246.0,0.304890,0.304890
247,fusionx-v2,FusionX V2,NaN,None,,https://fusionx.finance/swap?,https://assets.coingecko.com/markets/images/11...,False,5.0,247.0,0.273395,0.273395
248,spiritswap,SpiritSwap,2021.0,None,SpiritSwap is a decentralized exchange (DEX) o...,https://app.spiritswap.finance/#/,https://assets.coingecko.com/markets/images/67...,False,5.0,248.0,0.267439,0.267439


In [106]:
df_exchanges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               250 non-null    object 
 1   name                             250 non-null    object 
 2   year_established                 142 non-null    float64
 3   country                          129 non-null    object 
 4   description                      245 non-null    object 
 5   url                              250 non-null    object 
 6   image                            250 non-null    object 
 7   has_trading_incentive            247 non-null    object 
 8   trust_score                      249 non-null    float64
 9   trust_score_rank                 249 non-null    float64
 10  trade_volume_24h_btc             250 non-null    float64
 11  trade_volume_24h_btc_normalized  250 non-null    float64
dtypes: float64(5), object(

In [107]:
df_exchanges['has_trading_incentive'].unique()

array([False, None], dtype=object)

In [108]:
# Elimina las columnas con datos no relevantes
df_exchanges.drop(['url','image','description','has_trading_incentive'],axis=1,inplace=True)

In [109]:
df_exchanges[['trade_volume_24h_btc','trade_volume_24h_btc_normalized']] = df_exchanges[['trade_volume_24h_btc','trade_volume_24h_btc_normalized']].round(2)

In [110]:
df_exchanges[['year_established','trust_score_rank','trust_score']] = df_exchanges[['year_established','trust_score_rank','trust_score']].astype('Int64')

In [111]:
df_exchanges.head()

,id,name,year_established,country,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,binance,Binance,2017,Cayman Islands,10,1,200455.72,131112.82
1,gdax,Coinbase Exchange,2012,United States,10,2,30308.93,30308.93
2,bybit_spot,Bybit,2018,British Virgin Islands,10,3,28728.84,28728.84
3,huobi,Huobi,2013,Seychelles,10,4,38706.51,19743.02
4,kraken,Kraken,2011,United States,10,5,18114.27,13082.62


In [112]:
df_exchanges.to_csv('./Data/exchanges.csv',index=False)